In [8]:
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv


load_dotenv() 


user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = 'localhost'  
port = '5432'
database = os.getenv('POSTGRES_DB')  


connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)


print("Connecting to database...")
with engine.connect() as conn:
    result = conn.execute(text("SELECT current_database(), current_user"))
    db, user_connected = result.fetchone()
    print(f"Connected to database successfully")

Connecting to database...
Connected to database successfully


In [2]:
import pandas as pd

In [9]:


query = """
SELECT * 
FROM orchestration.ingestion_checkpoint
"""

try:
    df = pd.read_sql(query, engine)
    
    print(f"Query executed successfully!")
    print(f"Total rows retrieved: {len(df)}")
    print(f"Columns in table: {list(df.columns)}")
    
    print("\n👀 First 5 rows of the data:")
    print(df.head())
    
except Exception as e:
    print(f"Error executing query: {e}")

Query executed successfully!
Total rows retrieved: 96
Columns in table: ['id', 'year_month', 'service_type', 'status', 'started_at', 'completed_at', 'rows_ingested', 'error_message', 'retry_count', 'created_at', 'updated_at']

👀 First 5 rows of the data:
    id year_month service_type     status                 started_at  \
0   61    2024-01       yellow  completed 2026-02-19 05:15:58.903994   
1    3    2025-01       yellow  completed 2026-02-19 15:36:53.646667   
2    5    2025-02       yellow  completed 2026-02-19 15:48:42.954276   
3  124    2022-11       yellow  completed 2026-02-20 10:05:00.891962   
4  188    2024-04       yellow  completed 2026-02-20 16:57:57.000263   

                completed_at  rows_ingested  \
0 2026-02-19 15:32:43.248990      2964624.0   
1 2026-02-19 15:45:34.318507      3475226.0   
2 2026-02-19 15:57:22.014805      3577543.0   
3 2026-02-20 10:28:22.775792      3252717.0   
4 2026-02-20 17:23:21.031802      3514289.0   

                             

In [ ]:

query_q1 = """
SELECT 
    d.year_month,
    d.month_name,
    COUNT(*) AS trip_count
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month
ORDER BY d.month;
"""

try:
    df_q1 = pd.read_sql(query_q1, engine)
    print("Question 1 - Viajes por mes (2024)")
    print("=" * 50)
    print(f"Total rows: {len(df_q1)}")
    print("\nResults:")
    display(df_q1)
    

except Exception as e:
    print(f" Error: {e}")

Question 1 - Viajes por mes (2024)
Total rows: 12

Results:


,year_month,month_name,trip_count
0,2024-01,January,2923396
1,2024-02,February,2952275
2,2024-03,March,3494708
3,2024-04,April,3467797
4,2024-05,May,3675873
5,2024-06,June,3480521
6,2024-07,July,3022881
7,2024-08,August,2916491
8,2024-09,September,3535781
9,2024-10,October,3735637


### 1. Viajes por mes (2024): count(*) por month
Octubre fue el mes con mayor cantidad de viajes en 2024 (3.74M), mientras que agosto registró la menor actividad (2.92M). Se observa un patrón estacional con picos en mayo y octubre, y una caída durante los meses de verano (julio-agosto).

In [6]:

query_q2 = """
SELECT 
    d.year_month,
    d.month_name,
    st.service_type_name,
    COUNT(*) AS trip_count
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
JOIN analytics_gold.dim_service_type st ON f.service_type_key = st.service_type_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month, st.service_type_name, st.service_type_key
ORDER BY d.month, st.service_type_key;
"""

try:
    df_q2 = pd.read_sql(query_q2, engine)
    print("Question 2 - Viajes por service_type y mes (2024)")
    print("=" * 50)
    print(f"Total rows: {len(df_q2)}")
    print("\nResults:")
    display(df_q2)
    

except Exception as e:
    print(f"Error: {e}")

Question 2 - Viajes por service_type y mes (2024)
Total rows: 24

Results:


,year_month,month_name,service_type_name,trip_count
0,2024-01,January,Yellow Taxi,2869850
1,2024-01,January,Green Taxi,53546
2,2024-02,February,Yellow Taxi,2901664
3,2024-02,February,Green Taxi,50611
4,2024-03,March,Yellow Taxi,3440413
5,2024-03,March,Green Taxi,54295
6,2024-04,April,Yellow Taxi,3414667
7,2024-04,April,Green Taxi,53130
8,2024-05,May,Yellow Taxi,3618154
9,2024-05,May,Green Taxi,57719


### 2. Viajes por service_type y mes
Yellow Taxi domina completamente el mercado representando aproximadamente el 98.5% de todos los viajes, mientras que Green Taxi solo contribuye con el 1.5% restante. Esta proporción se mantiene consistente a lo largo de todo el año. El mes con mayor actividad para ambos servicios fue octubre, mientras que agosto (para Yellow) y julio (para Green) registraron los volúmenes más bajos.

In [7]:
query_q3 = """
SELECT 
    z.zone_name,
    z.borough,
    COUNT(*) AS trip_count
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE d.year = 2024
GROUP BY z.zone_name, z.borough, z.zone_key
ORDER BY trip_count DESC
LIMIT 10;
"""

try:
    df_q3 = pd.read_sql(query_q3, engine)
    print(" Question 3 - Top 10 zonas de pickup (2024)")
    print("=" * 50)
    print(f"Total rows: {len(df_q3)}")
    print("\n Results:")
    display(df_q3)
    
   
except Exception as e:
    print(f" Error: {e}")

 Question 3 - Top 10 zonas de pickup (2024)
Total rows: 10

 Results:


,zone_name,borough,trip_count
0,JFK Airport,Queens,1872278
1,Upper East Side South,Manhattan,1872240
2,Midtown Center,Manhattan,1862305
3,Upper East Side North,Manhattan,1695939
4,Midtown East,Manhattan,1381901
5,Times Sq/Theatre District,Manhattan,1339049
6,Penn Station/Madison Sq West,Manhattan,1325040
7,Lincoln Square East,Manhattan,1281704
8,LaGuardia Airport,Queens,1261721
9,Murray Hill,Manhattan,1140514


### 3. Top 10 zonas de pickup (total 2024).
JFK Airport lidera como la zona con más viajes iniciados en 2024, seguida muy de cerca por Upper East Side South. Manhattan domina el ranking con 8 de las 10 zonas, concentrando la mayor actividad de viajes. Los dos aeropuertos (JFK y LaGuardia) en Queens son las únicas zonas fuera de Manhattan en el top 10, reflejando la alta demanda de viajes desde/hacia aeropuertos.

In [8]:
query_q4 = """
SELECT 
    z.zone_name,
    z.borough,
    COUNT(*) AS trip_count
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
JOIN analytics_gold.dim_zones z ON f.do_zone_key = z.zone_key
WHERE d.year = 2024
GROUP BY z.zone_name, z.borough, z.zone_key
ORDER BY trip_count DESC
LIMIT 10;
"""

try:
    df_q4 = pd.read_sql(query_q4, engine)
    print(" Question 4 - Top 10 zonas de dropoff (2024)")
    print("=" * 50)
    print(f"Total rows: {len(df_q4)}")
    print("\n Results:")
    display(df_q4)
except Exception as e:
    print(f" Error: {e}")

 Question 4 - Top 10 zonas de dropoff (2024)
Total rows: 10

 Results:


,zone_name,borough,trip_count
0,Upper East Side North,Manhattan,1787861
1,Upper East Side South,Manhattan,1694421
2,Midtown Center,Manhattan,1497989
3,Times Sq/Theatre District,Manhattan,1259373
4,Murray Hill,Manhattan,1179707
5,Midtown East,Manhattan,1152593
6,Lincoln Square East,Manhattan,1119977
7,Upper West Side South,Manhattan,1119510
8,Lenox Hill West,Manhattan,1044270
9,East Chelsea,Manhattan,1042739


### 4. Top 10 zonas de dropoff (total 2024).
A diferencia de las zonas de pickup donde los aeropuertos dominaban, las zonas de dropoff están exclusivamente en Manhattan. Upper East Side North es la zona con más llegadas, seguida por Upper East Side South. Esto sugiere que Manhattan es el principal destino de viajes, particularmente el Upper East Side y áreas centrales como Midtown y Times Square. Los aeropuertos no aparecen en el top 10 de destinos, indicando que la mayoría de los viajes desde aeropuertos tienen destino en Manhattan.

In [9]:
query_q5 = """
WITH monthly_borough_counts AS (
    SELECT 
        d.year_month,
        d.month_name,
        z.borough,
        COUNT(*) AS trip_count,
        ROW_NUMBER() OVER (PARTITION BY d.year_month ORDER BY COUNT(*) DESC) AS rank
    FROM analytics_gold.fct_trips f
    JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
    JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
    WHERE d.year = 2024
    GROUP BY d.year_month, d.month_name, d.month, z.borough
)
SELECT 
    year_month,
    month_name,
    borough,
    trip_count,
    rank
FROM monthly_borough_counts
WHERE rank <= 5
ORDER BY year_month, rank;
"""

try:
    df_q5 = pd.read_sql(query_q5, engine)
    print(" Question 5 - Top 5 boroughs por mes (pickup) 2024")
    print("=" * 50)
    print(f"Total rows: {len(df_q5)}")
    print("\n Results (first 15 rows shown):")
    display(df_q5.head(15))
except Exception as e:
    print(f" Error: {e}")

 Question 5 - Top 5 boroughs por mes (pickup) 2024
Total rows: 60

 Results (first 15 rows shown):


,year_month,month_name,borough,trip_count,rank
0,2024-01,January,Manhattan,2605735,1
1,2024-01,January,Queens,271552,2
2,2024-01,January,Brooklyn,29224,3
3,2024-01,January,Unknown,9683,4
4,2024-01,January,Bronx,6383,5
5,2024-02,February,Manhattan,2655456,1
6,2024-02,February,Queens,248993,2
7,2024-02,February,Brooklyn,31749,3
8,2024-02,February,Unknown,9055,4
9,2024-02,February,Bronx,6352,5


### 5. Top 5 boroughs por mes (pickup)
Manhattan domina todos los meses con aproximadamente el 88-90% de los viajes. Queens se mantiene consistentemente en segundo lugar (8-9%), seguido por Brooklyn (1-1.5%). El Bronx y Staten Island (no visible en el top 5 de algunos meses) tienen participación marginal. La categoría "Unknown" representa zonas sin borough asignado, posiblemente datos faltantes o erróneos. La distribución se mantiene estable durante todo el año, con Manhattan como el epicentro indiscutido de viajes en taxi.

In [ ]:
query_q6 = """
WITH valid_trips AS (
    -- Get valid trips from raw table with timestamps
    SELECT 
        b.pickup_ts,
        b.dropoff_ts,
        EXTRACT(DOW FROM b.pickup_ts) AS day_of_week,
        EXTRACT(HOUR FROM b.pickup_ts) AS hour,
        b.trip_distance,
        b.total_amount
    FROM analytics_bronze.stg_taxi_trips_bronze b
    WHERE EXTRACT(YEAR FROM b.pickup_ts) = 2024
      -- Quality filters
      AND b.pickup_ts IS NOT NULL 
      AND b.dropoff_ts IS NOT NULL
      AND b.pickup_ts <= b.dropoff_ts
      AND b.trip_distance >= 0
      AND b.total_amount >= 0
      AND b.passenger_count >= 0
      AND b.fare_amount >= 0
),
hourly_stats AS (
    SELECT 
        vt.day_of_week,
        d.day_name,
        vt.hour,
        COUNT(*) AS trip_count,
        ROUND(AVG(vt.trip_distance)::NUMERIC, 2) AS avg_distance,
        ROUND(AVG(vt.total_amount)::NUMERIC, 2) AS avg_amount,
        RANK() OVER (PARTITION BY vt.day_of_week ORDER BY COUNT(*) DESC) AS rank_by_day
    FROM valid_trips vt
    JOIN analytics_gold.dim_date d ON vt.day_of_week = d.day_of_week
    GROUP BY vt.day_of_week, d.day_name, vt.hour
)
SELECT 
    day_name,
    hour AS peak_hour,
    trip_count,
    avg_distance,
    avg_amount,
    ROUND(100.0 * trip_count / SUM(trip_count) OVER (PARTITION BY day_name), 2) AS pct_of_daily_trips
FROM hourly_stats
WHERE rank_by_day <= 5
ORDER BY 
    CASE day_name
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END,
    rank_by_day;
"""

try:
    df_q6 = pd.read_sql(query_q6, engine)
    print("Question 6 - Horas pico (top 5 horas) para cada día de semana (2024)")
    print("=" * 70)
    print(f"Total rows: {len(df_q6)}")
    print("\nResults - Top 5 peak hours for each day of week:")
    display(df_q6)
    
    print("\nSummary by time of day:")
    summary_q6 = """
    SELECT 
        CASE 
            WHEN hour BETWEEN 6 AND 9 THEN 'Morning Rush (6-9)'
            WHEN hour BETWEEN 10 AND 15 THEN 'Midday (10-15)'
            WHEN hour BETWEEN 16 AND 19 THEN 'Evening Rush (16-19)'
            WHEN hour BETWEEN 20 AND 23 THEN 'Night (20-23)'
            ELSE 'Late Night (0-5)'
        END AS time_block,
        COUNT(DISTINCT day_name) AS days_appeared,
        SUM(trip_count) AS total_trips,
        ROUND(AVG(avg_distance), 2) AS avg_distance,
        ROUND(AVG(avg_amount), 2) AS avg_amount
    FROM (
        SELECT * FROM hourly_stats WHERE rank_by_day <= 5
    ) top_hours
    GROUP BY 
        CASE 
            WHEN hour BETWEEN 6 AND 9 THEN 'Morning Rush (6-9)'
            WHEN hour BETWEEN 10 AND 15 THEN 'Midday (10-15)'
            WHEN hour BETWEEN 16 AND 19 THEN 'Evening Rush (16-19)'
            WHEN hour BETWEEN 20 AND 23 THEN 'Night (20-23)'
            ELSE 'Late Night (0-5)'
        END
    ORDER BY total_trips DESC;
    """
    df_summary = pd.read_sql(summary_q6, engine)
    display(df_summary)
    
except Exception as e:
    print(f"Error: {e}")

### 6. Horas pico (top 5 horas) para cada día de semana

In [11]:
query_q7 = """
SELECT 
    d.day_name,
    COUNT(*) AS trip_count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) AS percentage,
    RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024
GROUP BY d.day_name, d.day_of_week
ORDER BY rank;
"""

try:
    df_q7 = pd.read_sql(query_q7, engine)
    print(" Question 7 - Distribución de viajes por día de semana (ranking)")
    print("=" * 50)
    print(f"Total rows: {len(df_q7)}")
    print("\n Results:")
    display(df_q7)
except Exception as e:
    print(f" Error: {e}")

 Question 7 - Distribución de viajes por día de semana (ranking)
Total rows: 7

 Results:


,day_name,trip_count,percentage,rank
0,Thursday,6302951,15.63,1
1,Saturday,6093042,15.11,2
2,Wednesday,6008682,14.90,3
3,Friday,5963434,14.78,4
4,Tuesday,5793470,14.36,5
5,Sunday,5151197,12.77,6
6,Monday,5022280,12.45,7


### 7. Distribución de viajes por día de semana (ranking)
Jueves es el día con mayor actividad (15.63% de los viajes), seguido por Sábado y Miércoles. Los días de mayor actividad son jueves-viernes-sábado, mientras que Domingo y Lunes son los días más flojos con ~12.5% cada uno. Esto sugiere que los viajes en taxi se concentran en los días de salidas nocturnas (jueves a sábado) y disminuyen al inicio de la semana laboral. La distribución es relativamente pareja entre semana, con una caída notable los domingos y lunes.

In [12]:
query_q8 = """
SELECT 
    d.year_month,
    d.month_name,
    SUM(f.total_amount) AS total_revenue
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month
ORDER BY d.month;
"""

try:
    df_q8 = pd.read_sql(query_q8, engine)
    print(" Question 8 - Ingreso total (total_amount) por mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q8)}")
    print("\n Results:")
    display(df_q8)
except Exception as e:
    print(f" Error: {e}")

 Question 8 - Ingreso total (total_amount) por mes
Total rows: 12

 Results:


,year_month,month_name,total_revenue
0,2024-01,January,7.967123e+07
1,2024-02,February,8.017779e+07
2,2024-03,March,9.708954e+07
3,2024-04,April,9.749890e+07
4,2024-05,May,1.063204e+08
5,2024-06,June,9.965762e+07
6,2024-07,July,8.741262e+07
7,2024-08,August,8.510150e+07
8,2024-09,September,1.041208e+08
9,2024-10,October,1.095449e+08


### 8. Ingreso total (total_amount) por mes
Octubre fue el mes con mayor ingreso ($109.5M), seguido por Mayo ($106.3M) y Septiembre ($104.1M). Los meses de verano (julio-agosto) muestran una caída significativa en ingresos, consistente con la menor cantidad de viajes en ese período. El ingreso mensual promedio en 2024 fue de aproximadamente $96M. La tendencia sigue el mismo patrón que los viajes, con un crecimiento hacia fin de año y una recuperación notable en septiembre-octubre.

In [13]:
query_q9 = """
SELECT 
    d.year_month,
    d.month_name,
    st.service_type_name,
    SUM(f.total_amount) AS total_revenue
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
JOIN analytics_gold.dim_service_type st ON f.service_type_key = st.service_type_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month, st.service_type_name, st.service_type_key
ORDER BY d.month, st.service_type_key;
"""

try:
    df_q9 = pd.read_sql(query_q9, engine)
    print(" Question 9 - Ingreso total por service_type y mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q9)}")
    print("\n Results:")
    display(df_q9)
except Exception as e:
    print(f" Error: {e}")

 Question 9 - Ingreso total por service_type y mes
Total rows: 24

 Results:


,year_month,month_name,service_type_name,total_revenue
0,2024-01,January,Yellow Taxi,7.847889e+07
1,2024-01,January,Green Taxi,1.192335e+06
2,2024-02,February,Yellow Taxi,7.903716e+07
3,2024-02,February,Green Taxi,1.140633e+06
4,2024-03,March,Yellow Taxi,9.585104e+07
5,2024-03,March,Green Taxi,1.238500e+06
6,2024-04,April,Yellow Taxi,9.625929e+07
7,2024-04,April,Green Taxi,1.239605e+06
8,2024-05,May,Yellow Taxi,1.049025e+08
9,2024-05,May,Green Taxi,1.417888e+06


### 9. Ingreso total por service_type y mes
Yellow Taxi genera aproximadamente el 98.7% del ingreso total mensual, mientras que Green Taxi contribuye con solo el 1.3% restante ($1.1M - $1.4M por mes). Esta proporción es aún más pronunciada que en volumen de viajes, indicando que los viajes en Yellow Taxi tienen un ticket promedio ligeramente mayor. El pico de ingresos para ambos servicios ocurre en mayo y octubre, consistente con la estacionalidad observada. Diciembre muestra la mayor concentración de ingresos en Yellow Taxi (98.84%), posiblemente por viajes más largos o tarifas más altas en fin de año.

In [14]:
query_q10 = """
SELECT 
    d.year_month,
    d.month_name,
    ROUND(AVG(f.tip_percentage) * 100, 2) AS avg_tip_percentage
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024 AND f.fare_amount > 0
GROUP BY d.year_month, d.month_name, d.month
ORDER BY d.month;
"""

try:
    df_q10 = pd.read_sql(query_q10, engine)
    print(" Question 10 - tip % promedio por mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q10)}")
    print("\n Results:")
    display(df_q10)
except Exception as e:
    print(f" Error: {e}")

 Question 10 - tip % promedio por mes
Total rows: 12

 Results:


,year_month,month_name,avg_tip_percentage
0,2024-01,January,21.30
1,2024-02,February,20.55
2,2024-03,March,19.17
3,2024-04,April,19.21
4,2024-05,May,18.93
5,2024-06,June,18.66
6,2024-07,July,19.10
7,2024-08,August,18.83
8,2024-09,September,18.33
9,2024-10,October,19.27


### 10. tip % promedio por mes (avg(tip_amount / nullif(fare_amount,0)))
Enero registra el porcentaje de propina más alto del año (21.30%), posiblemente por generosidad post-navideña o menor volumen de viajes. El porcentaje de propina tiende a disminuir hacia mediados de año, con septiembre mostrando el valor más bajo (18.33%). Se observa un repunte en noviembre-diciembre, probablemente asociado a la temporada de fiestas. El promedio anual de propina se mantiene consistentemente entre 18-21%, mostrando que los neoyorquinos y visitantes son buenos propinadores independientemente de la época del año.

In [15]:
query_q11 = """
SELECT 
    d.year_month,
    d.month_name,
    z.borough,
    ROUND(AVG(f.tip_percentage) * 100, 2) AS avg_tip_percentage,
    COUNT(*) AS trip_count
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE d.year = 2024 AND f.fare_amount > 0
GROUP BY d.year_month, d.month_name, d.month, z.borough
ORDER BY d.month, avg_tip_percentage DESC;
"""

try:
    df_q11 = pd.read_sql(query_q11, engine)
    print(" Question 11 - tip % por borough y mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q11)}")
    print("\n Results:")
    display(df_q11)
except Exception as e:
    print(f" Error: {e}")

 Question 11 - tip % por borough y mes
Total rows: 96

 Results:


,year_month,month_name,borough,avg_tip_percentage,trip_count
0,2024-01,January,Manhattan,21.50,2605561
1,2024-01,January,Queens,21.29,271476
2,2024-01,January,Unknown,20.14,9681
3,2024-01,January,Unknown Borough,12.15,707
4,2024-01,January,EWR,11.63,49
...,...,...,...,...,...
91,2024-12,December,Unknown Borough,11.74,843
92,2024-12,December,EWR,11.38,149
93,2024-12,December,Brooklyn,5.94,62854
94,2024-12,December,Staten Island,1.54,140


### 11. tip % por borough y mes
Existe una marcada desigualdad geográfica en las propinas. Manhattan y Queens mantienen propinas consistentemente altas (18-21%) durante todo el año, reflejando viajes de turistas y negocios. Brooklyn muestra propinas moderadas (6-9%), mientras que Bronx y Staten Island tienen propinas significativamente más bajas (1-5%), posiblemente por viajes más cortos o menor poder adquisitivo. EWR (Newark Airport) mantiene propinas respetables (10-14%) a pesar de estar fuera de NYC. El patrón mensual se mantiene consistente: enero es el mes más generoso en casi todos los boroughs, mientras que septiembre-octubre tienden a ser más bajos.



In [16]:
query_q12 = """
SELECT 
    z.zone_name,
    z.borough,
    SUM(f.total_amount) AS total_revenue,
    COUNT(*) AS trip_count,
    ROUND(AVG(f.total_amount), 2) AS avg_revenue_per_trip
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
GROUP BY z.zone_name, z.borough, z.zone_key
ORDER BY total_revenue DESC
LIMIT 10;
"""

try:
    df_q12 = pd.read_sql(query_q12, engine)
    print(" Question 12 - Top 10 zonas por ingreso total (pickup)")
    print("=" * 50)
    print(f"Total rows: {len(df_q12)}")
    print("\n Results:")
    display(df_q12)
except Exception as e:
    print(f" Error: {e}")

 Question 12 - Top 10 zonas por ingreso total (pickup)
Total rows: 10

 Results:


,zone_name,borough,total_revenue,trip_count,avg_revenue_per_trip
0,JFK Airport,Queens,1.549281e+08,1872278,82.75
1,LaGuardia Airport,Queens,8.649245e+07,1261721,68.55
2,Midtown Center,Manhattan,4.702582e+07,1862305,25.25
3,Upper East Side South,Manhattan,3.916046e+07,1872240,20.92
4,Times Sq/Theatre District,Manhattan,3.883707e+07,1339049,29.00
5,Upper East Side North,Manhattan,3.583399e+07,1695939,21.13
6,Midtown East,Manhattan,3.397669e+07,1381901,24.59
7,Penn Station/Madison Sq West,Manhattan,3.390455e+07,1325040,25.59
8,Midtown North,Manhattan,2.872645e+07,1136753,25.27
9,Lincoln Square East,Manhattan,2.864760e+07,1281704,22.35


### 12. Top 10 zonas por ingreso total (pickup)
Los aeropuertos dominan en ingresos totales a pesar de tener menor volumen de viajes que zonas de Manhattan. JFK Airport genera más del doble que cualquier zona de Manhattan ($155M), gracias a un ticket promedio muy superior ($83 vs $20-29). LaGuardia Airport le sigue en segundo lugar con $86.5M. Esto refleja que los viajes desde aeropuertos son significativamente más largos y costosos. En contraste, zonas de Manhattan como Midtown y Upper East Side generan ingresos altos principalmente por su enorme volumen de viajes, pero con tickets promedio mucho más bajos.

In [17]:
query_q13 = """
SELECT 
    z.zone_name,
    z.borough,
    ROUND(AVG(f.tip_percentage) * 100, 2) AS avg_tip_percentage,
    COUNT(*) AS trip_count,
    ROUND(AVG(f.fare_amount), 2) AS avg_fare,
    ROUND(AVG(f.tip_amount), 2) AS avg_tip
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024 
  AND f.fare_amount > 0
  AND f.tip_amount >= 0
GROUP BY z.zone_name, z.borough, z.zone_key
HAVING COUNT(*) >= 1000  -- Minimum trip threshold
ORDER BY avg_tip_percentage DESC
LIMIT 10;
"""

try:
    df_q13 = pd.read_sql(query_q13, engine)
    print(" Question 13 - Top 10 zonas por tip % (pickup) con mínimo 1000 viajes")
    print("=" * 50)
    print(f"Total rows: {len(df_q13)}")
    print("\n Results:")
    display(df_q13)
    print("\n Nota: Se usó un mínimo de 1000 viajes para asegurar significancia estadística.")
except Exception as e:
    print(f" Error: {e}")

 Question 13 - Top 10 zonas por tip % (pickup) con mínimo 1000 viajes
Total rows: 10

 Results:


,zone_name,borough,avg_tip_percentage,trip_count,avg_fare,avg_tip
0,South Ozone Park,Queens,103.59,14624,53.16,5.78
1,Baisley Park,Queens,81.33,14094,59.33,7.27
2,Outside of NYC,Unknown Borough,45.69,9383,70.83,6.72
3,Newark Airport,EWR,30.03,1240,78.84,10.30
4,Steinway,Queens,26.62,14322,32.03,3.64
5,Kips Bay,Manhattan,22.28,421343,15.19,2.42
6,Upper East Side South,Manhattan,22.20,1872063,13.43,2.66
7,Upper East Side North,Manhattan,21.62,1695718,13.69,2.68
8,LaGuardia Airport,Queens,21.55,1261580,44.16,9.13
9,Upper West Side South,Manhattan,21.48,1111818,14.51,2.80



 Nota: Se usó un mínimo de 1000 viajes para asegurar significancia estadística.


### 13. Top 10 zonas por tip % (pickup) con mínimo N viajes (define N y documenta)
Las zonas con mayor porcentaje de propina no son las más turísticas o exclusivas, sino áreas en Queens como South Ozone Park (103.6%) y Baisley Park (81.3%), donde la propina promedio supera el monto de la tarifa. Esto sugiere viajes muy cortos con tarifas bajas pero propinas fijas generosas, o posiblemente errores de registro.

Newark Airport y LaGuardia muestran propinas sólidas (~30% y 21.5%) consistentes con viajes aeroportuarios. Manhattan, a pesar de tener el mayor volumen, presenta porcentajes de propina más moderados (21-22%) debido a tarifas base más bajas pero propinas en dólares similares a otras zonas.

El caso de "Outside of NYC" (45.7%) probablemente representa viajes largos con tarifas altas donde la propina porcentual se calcula sobre montos mayores.

In [18]:
query_q14 = """
SELECT 
    pt.payment_type_name,
    pt.partition_group AS payment_category,
    COUNT(*) AS trip_count,
    ROUND(SUM(f.total_amount)::NUMERIC, 2) AS total_revenue,
    ROUND(AVG(f.total_amount)::NUMERIC, 2) AS avg_ticket,
    ROUND(AVG(f.tip_percentage) * 100, 2) AS avg_tip_percentage,
    ROUND(SUM(f.tip_amount)::NUMERIC, 2) AS total_tips,
    ROUND((SUM(f.tip_amount) / NULLIF(SUM(f.fare_amount), 0) * 100)::NUMERIC, 2) AS overall_tip_rate
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_payment_type pt ON f.payment_type_key = pt.payment_type_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
  AND pt.partition_group IN ('cash', 'card')
GROUP BY pt.payment_type_name, pt.partition_group
ORDER BY trip_count DESC;
"""

try:
    df_q14 = pd.read_sql(query_q14, engine)
    print(" Question 14 - Comparación cash vs card")
    print("=" * 50)
    print(f"Total rows: {len(df_q14)}")
    print("\n Results:")
    display(df_q14)
except Exception as e:
    print(f" Error: {e}")

 Question 14 - Comparación cash vs card
Total rows: 2

 Results:


,payment_type_name,payment_category,trip_count,total_revenue,avg_ticket,avg_tip_percentage,total_tips,overall_tip_rate
0,Credit Card,card,30615454,9.080363e+08,29.66,25.13,1.329643e+08,22.05
1,Cash,cash,5453455,1.352496e+08,24.80,0.00,1.324860e+03,0.00


### 14. Comparación cash vs card: viajes, ingreso total, tip %
El pago con tarjeta domina el mercado representando casi el 85% de los viajes y generando 6.7 veces más ingresos que el efectivo. Los viajes pagados con tarjeta tienen un ticket promedio significativamente mayor ($29.66 vs $24.80), lo que sugiere que viajes más largos o costosos tienden a pagarse con tarjeta.

La diferencia más dramática está en las propinas: los pagos con tarjeta generan prácticamente la totalidad de las propinas ($133M vs apenas $1,300 en efectivo). Esto refleja tanto la conveniencia de dejar propina al pagar con tarjeta como la dificultad de registrar propinas en efectivo en los datos.

Los pasajeros que pagan con tarjeta dejan un generoso 22% de propina sobre la tarifa, mientras que los pagos en efectivo efectivamente no registran propinas en el sistema (posiblemente no capturadas en los datos).

In [19]:
query_q15 = """
SELECT 
    d.year_month,
    d.month_name,
    ROUND(AVG(f.trip_time_minutes)::NUMERIC, 2) AS avg_duration_minutes
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month
ORDER BY d.month;
"""

try:
    df_q15 = pd.read_sql(query_q15, engine)
    print(" Question 15 - Duración promedio (min) por mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q15)}")
    print("\n Results:")
    display(df_q15)
except Exception as e:
    print(f" Error: {e}")

 Question 15 - Duración promedio (min) por mes
Total rows: 12

 Results:


,year_month,month_name,avg_duration_minutes
0,2024-01,January,15.80
1,2024-02,February,16.17
2,2024-03,March,16.85
3,2024-04,April,17.21
4,2024-05,May,18.22
5,2024-06,June,17.77
6,2024-07,July,17.42
7,2024-08,August,17.59
8,2024-09,September,18.82
9,2024-10,October,18.48


### 15. Duración promedio (min) por mes
La duración de los viajes aumenta progresivamente a lo largo del año. Enero tiene los viajes más cortos (15.8 min promedio), mientras que diciembre registra la mayor duración (18.9 min), un aumento del 19.7% respecto a enero.

Esto tal vez se puede epxlicar por lo siguiente:

Estacionalidad: Mayor congestión de tráfico en meses de fin de año (compras navideñas, eventos)

Clima: Condiciones invernales que ralentizan el tráfico en diciembre

Turismo: Mayor afluencia de visitantes en otoño que realizan viajes más largos

Septiembre-octubre también muestran duraciones elevadas (18.5 min), posiblemente por el regreso a la rutina laboral y escolar. Los meses de verano (junio-agosto) mantienen duraciones intermedias alrededor de 17.5-17.8 minutos.

In [20]:
query_q16 = """
SELECT 
    d.year_month,
    d.month_name,
    ROUND(AVG(f.trip_distance)::NUMERIC, 2) AS avg_distance_miles
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_date d ON f.pickup_date_key = d.date_key
WHERE d.year = 2024
GROUP BY d.year_month, d.month_name, d.month
ORDER BY d.month;
"""

try:
    df_q16 = pd.read_sql(query_q16, engine)
    print(" Question 16 - Distancia promedio por mes")
    print("=" * 50)
    print(f"Total rows: {len(df_q16)}")
    print("\n Results:")
    display(df_q16)
except Exception as e:
    print(f" Error: {e}")

 Question 16 - Distancia promedio por mes
Total rows: 12

 Results:


,year_month,month_name,avg_distance_miles
0,2024-01,January,4.25
1,2024-02,February,4.21
2,2024-03,March,4.78
3,2024-04,April,5.36
4,2024-05,May,5.51
5,2024-06,June,5.42
6,2024-07,July,5.26
7,2024-08,August,5.30
8,2024-09,September,5.99
9,2024-10,October,5.30


### 16. Distancia promedio por mes
La distancia de los viajes muestra una marcada estacionalidad. Los primeros meses del año (enero-febrero) tienen los viajes más cortos (4.2 millas), mientras que septiembre registra la mayor distancia promedio (6 millas), un incremento del 42% respecto a febrero.

Esto puede significar:

Viajes a aeropuertos: El aumento en distancia coincide con meses de mayor actividad de aeropuertos (primavera-otoño)

Turismo: Visitantes que realizan viajes más largos desde/hacia aeropuertos y zonas turísticas

Clima: Mejores condiciones climáticas en primavera y otoño que facilitan viajes más largos

La distancia promedio se mantiene elevada (5.2-5.6 millas) durante prácticamente todo el año excepto el primer bimestre, indicando que los viajes cortos dominan solo en temporada baja de invierno.

In [9]:
query_q17 = """
SELECT 
    z.borough,
    ROUND(AVG(f.speed_mph)::NUMERIC, 2) AS avg_speed_mph,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY f.speed_mph)::NUMERIC, 2) AS median_speed_mph,
    COUNT(*) AS trip_count,
    ROUND(MIN(f.speed_mph)::NUMERIC, 2) AS min_speed_mph,
    ROUND(MAX(f.speed_mph)::NUMERIC, 2) AS max_speed_mph
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
  AND f.speed_mph IS NOT NULL
  AND f.speed_mph BETWEEN 0 AND 100 -- Remove unrealistic speeds
  AND f.trip_time_minutes > 0
  AND f.trip_distance > 0
GROUP BY z.borough
HAVING COUNT(*) >= 10
ORDER BY 
    CASE z.borough
        WHEN 'Manhattan' THEN 1
        WHEN 'Brooklyn' THEN 2
        WHEN 'Queens' THEN 3
        WHEN 'Bronx' THEN 4
        WHEN 'Staten Island' THEN 5
        ELSE 6
    END;
"""

try:
    df_q17 = pd.read_sql(query_q17, engine)
    print("Question 17 - Velocidad promedio (mph) por borough")
    print("=" * 50)
    print(f"Total rows: {len(df_q17)}")
    print("\nResults:")
    display(df_q17)
except Exception as e:
    print(f"Error: {e}")

Question 17 - Velocidad promedio (mph) por borough
Total rows: 8

Results:


,borough,avg_speed_mph,median_speed_mph,trip_count,min_speed_mph,max_speed_mph
0,Manhattan,9.72,8.84,35638391,0.00,100.00
1,Brooklyn,11.94,10.44,637669,0.00,99.95
2,Queens,21.50,20.51,3799728,0.00,100.00
3,Bronx,13.87,12.14,120489,0.00,99.57
4,Staten Island,25.26,24.60,1485,0.01,96.00
5,EWR,20.63,20.90,1162,0.05,98.63
6,Unknown,11.67,9.78,113660,0.00,100.00
7,Unknown Borough,25.54,22.50,8735,0.00,100.00


### 17. Velocidad promedio (mph) por borough y franja horaria
Manhattan es el borough con menor velocidad promedio (9.7 mph), reflejando la alta congestión vehicular, semáforos y tráfico denso característico de la zona. La mediana (8.8 mph) confirma que la mitad de los viajes en Manhattan no superan esta velocidad.
Brooklyn y Bronx mantienen velocidades intermedias (11-14 mph), consistentes con áreas urbanas densas pero con mejor flujo que Manhattan. EWR (Newark Airport) muestra alta velocidad (20.6 mph) por viajes desde/hacia autopistas.
Y por el otro lado, Queens y Staten Island cuentan con las velocidades más altas

In [22]:
query_q18 = """
SELECT 
    z.borough,
    COUNT(*) AS trip_count,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY f.trip_time_minutes)::NUMERIC, 2) AS p50_duration_min,
    ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY f.trip_time_minutes)::NUMERIC, 2) AS p90_duration_min
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
  AND f.trip_time_minutes > 0
GROUP BY z.borough
ORDER BY p90_duration_min DESC;
"""

try:
    df_q18 = pd.read_sql(query_q18, engine)
    print(" Question 18 - Percentiles p50 y p90 de duración por borough")
    print("=" * 50)
    print(f"Total rows: {len(df_q18)}")
    print("\n Results:")
    display(df_q18)
except Exception as e:
    print(f"Error: {e}")

 Question 18 - Percentiles p50 y p90 de duración por borough
Total rows: 8

 Results:


,borough,trip_count,p50_duration_min,p90_duration_min
0,Bronx,120847,28.73,73.12
1,Queens,3803491,33.87,63.15
2,Unknown Borough,9501,9.83,59.80
3,Brooklyn,638770,22.58,56.68
4,Staten Island,1501,21.13,50.73
5,Unknown,114155,13.00,35.15
6,EWR,1247,0.22,31.05
7,Manhattan,35645544,12.05,26.83


### 18. Percentiles p50 y p90 de duración por borough
Bronx y Queens tienen las duraciones más extremas en el percentil 90, con viajes que superan los 73 y 63 minutos respectivamente. Esto refleja viajes interborough o hacia zonas alejadas, posiblemente con tráfico intenso.

Manhattan tiene la duración más compacta (p90 de solo 26.8 minutos), indicando que incluso el 10% de los viajes más largos no supera la media hora. La mediana en Manhattan (12 minutos) confirma viajes predominantemente cortos dentro del borough.

Brooklyn muestra duraciones intermedias (p50: 22.6 min, p90: 56.7 min), consistente con viajes hacia/dede Manhattan o dentro del borough.

Dato llamativo: EWR muestra mediana de 0.22 minutos (13 segundos), probablemente por viajes muy cortos dentro del aeropuerto o errores de registro. El p90 de 31 minutos refleja viajes desde Newark hacia NYC.

La diferencia entre p50 y p90 es más pronunciada en Bronx (44 minutos de diferencia) y Queens (29 minutos), indicando alta variabilidad en la duración de viajes en estos boroughs.

In [5]:
query_q19 = """
SELECT 
    z.zone_name,
    z.borough,
    COUNT(*) AS trip_count,
    ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY f.trip_time_minutes)::NUMERIC, 2) AS p90_duration_min,
    ROUND(AVG(f.trip_time_minutes)::NUMERIC, 2) AS avg_duration_min,
    ROUND(MAX(f.trip_time_minutes)::NUMERIC, 2) AS max_duration_min
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones z ON f.pu_zone_key = z.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
  AND f.trip_time_minutes > 0
GROUP BY z.zone_name, z.borough, z.zone_key
HAVING COUNT(*) >= 100  -- Minimum trips for statistical significance
ORDER BY p90_duration_min DESC
LIMIT 10;
"""

try:
    df_q19 = pd.read_sql(query_q19, engine)
    print("Question 19 - Top 10 zonas (pickup) por p90 de duración")
    print("=" * 50)
    print(f"Total rows: {len(df_q19)}")
    print("\n Results:")
    display(df_q19)
except Exception as e:
    print(f" Error: {e}")

Question 19 - Top 10 zonas (pickup) por p90 de duración
Total rows: 10

 Results:


,zone_name,borough,trip_count,p90_duration_min,avg_duration_min,max_duration_min
0,Far Rockaway,Queens,3707,107.36,65.51,587.60
1,Hammels/Arverne,Queens,3694,105.07,60.57,900.00
2,Coney Island,Brooklyn,7353,103.00,63.51,1439.43
3,Rockaway Park,Queens,1220,96.73,49.26,185.10
4,Brighton Beach,Brooklyn,2917,95.28,50.76,1436.13
5,Gravesend,Brooklyn,2229,91.92,48.63,1425.07
6,Rosedale,Queens,2386,91.80,45.63,619.72
7,Co-Op City,Bronx,5408,90.50,48.63,473.22
8,Cambria Heights,Queens,2058,89.58,48.55,211.20
9,Laurelton,Queens,2180,89.03,50.17,262.58


### 19. Top 10 zonas (pickup) por p90 de duración
Las zonas con viajes más largos (p90) se concentran en los extremos de la ciudad: Far Rockaway (Queens), Coney Island (Brooklyn) y zonas costeras/alejadas. Esto tiene sentido pues son áreas periféricas donde los viajes hacia Manhattan u otros centros son necesariamente largos.

Far Rockaway lidera con un p90 de 107 minutos (casi 1.8 horas), indicando que el 10% de los viajes desde esta zona superan esta duración. Los valores máximos extremos (>900 minutos o 15 horas) sugieren posibles viajes interurbanos o errores de registro.

Queens domina el ranking con 7 de las 10 zonas, reflejando su gran extensión geográfica y la distancia a centros de actividad. Brooklyn aporta 3 zonas (Coney Island, Brighton Beach, Gravesend), todas en el extremo sur del borough.

La diferencia entre duración promedio y p90 (ej: Far Rockaway: 65 vs 107 min) indica una cola larga de viajes extremadamente largos que elevan el percentil 90 muy por encima del promedio.

In [8]:
query_q20 = """
SELECT 
    pu.borough AS pickup_borough,
    do_zone.borough AS dropoff_borough,
    COUNT(*) AS trip_count,
    ROUND(AVG(f.total_amount)::NUMERIC, 2) AS avg_revenue,
    ROUND(AVG(f.trip_distance)::NUMERIC, 2) AS avg_distance_miles,
    ROUND(AVG(f.trip_time_minutes)::NUMERIC, 2) AS avg_duration_min
FROM analytics_gold.fct_trips f
JOIN analytics_gold.dim_zones pu ON f.pu_zone_key = pu.zone_key
JOIN analytics_gold.dim_zones do_zone ON f.do_zone_key = do_zone.zone_key
WHERE EXTRACT(YEAR FROM f.pickup_date) = 2024
GROUP BY pu.borough, do_zone.borough
ORDER BY trip_count DESC
LIMIT 10;
"""

try:
    df_q20 = pd.read_sql(query_q20, engine)
    print("Question 20 - Top 10 rutas borough→borough por número de viajes")
    print("=" * 60)
    print(f"Total rows: {len(df_q20)}")
    print("\nResults:")
    display(df_q20)
except Exception as e:
    print(f"Error: {e}")

Question 20 - Top 10 rutas borough→borough por número de viajes
Total rows: 10

Results:


,pickup_borough,dropoff_borough,trip_count,avg_revenue,avg_distance_miles,avg_duration_min
0,Manhattan,Manhattan,33452361,21.48,3.28,13.94
1,Queens,Manhattan,2131707,81.81,15.02,43.53
2,Manhattan,Queens,1087301,68.89,14.37,38.95
3,Queens,Queens,910202,39.18,10.50,20.58
4,Manhattan,Brooklyn,744251,44.44,7.13,31.04
5,Queens,Brooklyn,560931,69.21,14.88,40.05
6,Brooklyn,Brooklyn,354289,22.85,13.98,22.80
7,Brooklyn,Manhattan,196206,40.01,32.91,36.38
8,Manhattan,Bronx,121550,44.22,12.55,33.66
9,Queens,Unknown Borough,109962,133.19,27.83,45.81


### 20. Top 10 rutas borough→borough (pickup borough to dropoff borough) por número de viajes
La ruta intra-Manhattan domina absolutamente con 33.4 millones de viajes (85% del total), reflejando que la mayoría de los viajes comienzan y terminan dentro del mismo borough. Estas son rutas cortas (3.3 millas, 14 minutos) y de bajo ticket ($21.5).

Queens-Manhattan es la segunda ruta más importante en ambos sentidos, sumando 3.2 millones de viajes. Estas rutas tienen tickets muy superiores ($69-82) y distancias 4-5 veces mayores que las rutas intra-Manhattan, explicado por viajes aeroportuarios (JFK, LaGuardia).

Rutas inter-borough como Queens→Brooklyn y Manhattan→Brooklyn también muestran volúmenes significativos (>500k viajes cada una). Brooklyn→Brooklyn (354k viajes) indica actividad intra-borough relevante.

In [ ]:
print("=== fct_trips columns ===")
df_trips = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'fct_trips'
    ORDER BY ordinal_position;
""", engine)
display(df_trips)

print("\n=== dim_zones columns ===")
df_zones = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'dim_zones'
    ORDER BY ordinal_position;
""", engine)
display(df_zones)

print("\n=== dim_date columns ===")
df_date = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'dim_date'
    ORDER BY ordinal_position;
""", engine)
display(df_date)

print("\n=== dim_payment_type columns ===")
df_payment = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'dim_payment_type'
    ORDER BY ordinal_position;
""", engine)
display(df_payment)

print("\n=== dim_service_type columns ===")
df_service = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'dim_service_type'
    ORDER BY ordinal_position;
""", engine)
display(df_service)

print("\n=== dim_vendors columns ===")
df_vendors = pd.read_sql("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'analytics_gold' 
    AND table_name = 'dim_vendors'
    ORDER BY ordinal_position;
""", engine)
display(df_vendors)

print("\n=== Sample of fct_trips (first 5 rows) ===")
df_sample = pd.read_sql("""
    SELECT * FROM analytics_gold.fct_trips LIMIT 5;
""", engine)
display(df_sample)

=== fct_trips columns ===


,column_name,data_type
0,trip_key,character varying
1,pickup_date,date
2,pickup_date_key,integer
3,dropoff_date_key,integer
4,service_type_key,integer
5,vendor_key,integer
6,pu_zone_key,integer
7,do_zone_key,integer
8,payment_type_key,integer
9,passenger_count,integer



=== dim_zones columns ===


,column_name,data_type
0,zone_key,integer
1,borough,text
2,zone_name,text
3,service_zone,text



=== dim_date columns ===


...

,column_name,data_type
0,service_type_key,integer
1,service_type_name,character varying
2,service_type_code,character varying
3,partition_group,character varying



=== dim_vendors columns ===


,column_name,data_type
0,vendor_key,integer
1,vendor_id,integer
2,vendor_name,character varying
3,vendor_code,character varying
4,vendor_group,character varying
5,description,text
6,partition_group,character varying



=== Sample of fct_trips (first 5 rows) ===


,trip_key,pickup_date,pickup_date_key,dropoff_date_key,service_type_key,vendor_key,pu_zone_key,do_zone_key,payment_type_key,passenger_count,...,trip_time_minutes,tip_percentage,speed_mph,pickup_location_id,dropoff_location_id,payment_type_id,vendor_id,source_month,service_type,ingest_ts
0,1,2022-01-01,20220101,20220101,1,1,132,236,1,1,...,23.72,0.2427,48.32,132,236,1,1,2022-01,yellow,2026-02-20 04:23:50.974157
1,2,2022-01-01,20220101,20220101,1,1,7,7,2,1,...,4.75,0.0000,2.53,7,7,2,1,2022-01,yellow,2026-02-20 04:23:50.974157
2,3,2022-01-01,20220101,20220101,1,1,236,237,1,2,...,8.98,0.3588,9.35,236,237,1,1,2022-01,yellow,2026-02-20 04:23:50.974157
3,4,2022-01-01,20220101,20220101,1,1,239,262,1,1,...,10.62,0.2789,9.61,239,262,1,1,2022-01,yellow,2026-02-20 04:23:50.974157
4,5,2022-01-01,20220101,20220101,1,1,141,114,1,2,...,14.87,0.0556,21.39,141,114,1,1,2022-01,yellow,2026-02-20 04:23:50.974157
